In [133]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from scipy.stats import pmean

In [134]:
def clean_csv(root, f):
    print(f)

    df = pd.read_csv(os.path.join(root, f)).set_index('index')
    df.drop(columns=['even_stag', 'run', 'in_out', 'v_region'], inplace=True)
    # display(df)
    # display(df.T)

    new_cols = []
    cols = df.columns.to_list()
    for c in cols:
        pattern1 = re.compile(r';')
        col6 = re.split(pattern1, c)[-1]
        print(col6)

        pattern2 = re.compile(r'._\d_+')
        col6_clean = re.split(pattern2, col6)[-1]

        if col6_clean == '__':
            new_cols.append("Unassigned")
            continue

        new_cols.append(col6_clean)

    # print(new_cols)
    df.columns = new_cols
    deduped = df.groupby(lambda x:x, axis=1).sum()
    # display(deduped.T)

    return deduped.T


cleaned_df = []
for root, dirs, file in os.walk('/Volumes/TBHD/Bioinformatics/ion_torrent_qiime2_methods_manuscript/Classified_Tax_Counts_QIIME2_CSV_Files/Cutprimers/Species'):
    for f in file:
        if "Silva" in f:
            cleaned_df.append(clean_csv(root, f))

Species-Silva-V2-Cutprimers-Counts.csv
D_6__uncultured bacterium
D_6__uncultured bacterium
__
D_6__uncultured bacterium
D_6__uncultured bacterium
D_6__uncultured organism
__
__
__
D_6__Listeria monocytogenes
D_6__Staphylococcus aureus
D_6__uncultured bacterium
__
D_6__Enterococcus faecalis
D_6__Lactobacillus fermentum
D_6__uncultured bacterium
__
D_6__Streptococcus agalactiae
D_6__Streptococcus mutans
D_6__uncultured bacterium
__
D_6__uncultured bacterium
D_6__Rhodobacter sphaeroides
__
__
__
__
D_6__uncultured bacterium
__
__
__
D_6__Pseudomonas aeruginosa
__
__
Species-Silva-V3-Cutprimers-Counts.csv
D_6__uncultured bacterium
D_6__uncultured bacterium
D_6__uncultured bacterium
D_6__uncultured bacterium
D_6__uncultured organism
D_6__uncultured bacterium
__
D_6__uncultured bacterium
D_6__uncultured bacterium
__
__
D_6__Streptomyces sp.
__
D_6__Listeria monocytogenes
D_6__Staphylococcus aureus
D_6__uncultured bacterium
__
D_6__Enterococcus faecalis
D_6__Lactobacillus fermentum
D_6__uncul

In [139]:
def extra_column_cleaning(df):
    cols = df.columns.to_list()
    new_cols = []
    for c in cols:
        pattern = re.compile(r'_V\d')
        splitted = re.split(pattern, c)
        new_cols.append(splitted[0])

    df.columns = new_cols
    return df

extra_clean = [extra_column_cleaning(df) for df in cleaned_df]
# df1 = extra_clean[0].iloc[:, 0:8]
# display(df1)
# df2 = extra_clean[1].iloc[:, 0:8]
# df3 = extra_clean[2].iloc[:, 0:8]

# print(df1.shape)
# print(df2.shape)
# print(len(extra_clean))
out = pd.concat(extra_clean, axis=0)
# display(out)

groups = out.groupby(level=0, axis=0)
c = 0

final = pd.DataFrame()
for g in groups:
    # print(g[0])
    # display(g[1])
    m = pd.DataFrame(pmean(g[1], axis=0, p=1, nan_policy='omit')).T
    m.columns = g[1].columns
    m.index = [g[0]]

    final = pd.concat([final, m], axis=0)

    # display(m)
    c+=1

display(final)

,even_or_12,even_or_13,even_or_19,even_or_20,even_or_21,even_out_001,even_out_006,even_out_007,even_out_008,even_out_009,...,stag_out_011,stag_st_01,stag_st_02,stag_st_03,stag_st_04,stag_st_05,stag_st_06,stag_st_07,stag_st_08,stag_st_09
Acinetobacter baumannii,1723.000000,7419.250000,2610.000000,3118.25,2359.250000,113.500000,1537.750000,1542.000000,1668.250000,1486.250000,...,38.500000,44.000000,64.000000,31.250000,109.500000,94.250000,128.750000,123.750000,71.500000,38.500000
Bacillus cereus,1658.500000,8652.500000,2908.000000,3940.00,1851.000000,0.000000,2962.000000,3045.500000,3275.500000,5489.000000,...,146.500000,1110.500000,2191.500000,972.500000,3991.000000,2637.500000,3789.500000,3803.500000,2009.000000,896.500000
Bacillus subtilis,0.000000,0.000000,0.000000,0.00,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Enterococcus faecalis,1252.000000,5424.000000,2557.500000,2625.00,2111.500000,3.000000,4084.000000,3721.000000,5112.500000,3900.500000,...,4.500000,5.000000,12.500000,3.500000,15.500000,16.500000,28.000000,31.500000,8.000000,5.000000
Escherichia coli,0.000000,0.000000,0.000000,0.00,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
Lactobacillus fermentum,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Listeria monocytogenes,0.000000,0.000000,0.000000,0.00,0.000000,48.000000,0.000000,0.000000,0.000000,0.000000,...,52.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Neisseria meningitidis,1765.000000,8069.000000,2699.000000,3376.00,2630.000000,209.000000,1050.000000,1128.000000,1436.000000,1460.000000,...,57.000000,30.000000,37.000000,14.000000,46.000000,59.000000,70.000000,58.000000,32.000000,27.000000
Pseudomonas aeruginosa,1580.600000,7310.000000,2752.600000,3217.40,2380.200000,1464.000000,4637.200000,4326.600000,3357.200000,2656.400000,...,536.600000,706.400000,1323.800000,590.600000,2212.200000,1842.000000,2564.800000,2204.200000,1341.400000,557.200000
Rhodobacter sphaeroides,1394.666667,6419.000000,2114.333333,2567.00,1786.666667,11320.333333,1746.666667,2024.666667,1894.000000,900.000000,...,2590.333333,3499.000000,373.000000,2645.000000,741.666667,8283.666667,10187.666667,9886.666667,6348.333333,3012.666667
